In [3]:
# !pip install autokeras

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import autokeras as ak


In [3]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"


--2022-06-01 13:28:22--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203415 (199K) [application/x-httpd-php]
Saving to: ‘smsspamcollection.zip.1’

smsspamcollection.z 100%[===================>] 198.65K  1005KB/s    in 0.2s    

2022-06-01 13:28:23 (1005 KB/s) - ‘smsspamcollection.zip.1’ saved [203415/203415]



In [4]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [5]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [6]:
fileName = '/content/SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index=[])
with open(fileName) as f:
  for line in f.readlines():
    split = line.split('\t')
    df = df.append({'label': 1 if split[0]=='spam' else 0,
                    'text': split[1]},
                   ignore_index = True)
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
from sklearn.model_selection import train_test_split
x = np.array(df.text)
y = np.array(df.label)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)


In [ ]:
clf = ak.TextClassifier()
clf.fit(x_train, y_train, validation_split=0.3)

INFO:tensorflow:Reloading Oracle from existing project ./text_classifier/oracle.json
INFO:tensorflow:Reloading Tuner from ./text_classifier/tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
bert              |vanilla           |text_block_1/block_type
0                 |0                 |classification_head_1/dropout
adam_weight_decay |adam              |optimizer
2e-05             |0.001             |learning_rate
512               |None              |text_block_1/bert_block_1/max_sequence_length
20000             |5000              |text_block_1/max_tokens

Epoch 1/1000
61/92 [==================>...........] - ETA: 12:46 - loss: 0.1845 - accuracy: 0.9360

In [ ]:
from sklearn.metrics import classification_report
y_hat = clf.predict(x_test)
print(classification_report(y_test, y_hat))


In [ ]:
# get the best performing model
model = clf.export_model()
# display the architecture of the best performing model
model.summary()

In [ ]:
from tensorflow.keras.models import load_model
# save the model on disk
model.save("text_classifier", save_format = "tf")
# load the model from disk
loaded_model = load_model("text_classifier",
                          custom_objects = ak.CUSTOM_OBJECTS)
# use loaded model for future predictions on new samples
loaded_model.predict(x_test)